In [1]:
import  openpyxl 
import pandas as pd
import numpy as np
import os
import re


def checkVersion(workbook):
        cell_value = workbook['Info'].cell(row=4, column=4).value
        match = re.search(r"Version:\s*([\d.]+)\s*as", cell_value or "")
        version = float(match.group(1)[:3]) if match else None
        return version


def get_data_from_ranges(sheet, ranges):
    data = []
    for cell_range in ranges:
        if ":" not in cell_range:
            cell = sheet[cell_range]
            data.append({'Cell': cell.coordinate,'Count_values': 1 if cell.value is not None else 0})
        else:
            for row in sheet[cell_range]:
                for cell in row:
                    data.append({'Cell': cell.coordinate, 'Count_values': 1 if cell.value is not None else 0})
    return data


def get_ranges_for_version(version):
    if version in [2.4, 2.5]:
        return ['B11:B20','C8','F11:J20','H8']
    elif version ==2.6 :
        return ['B11:B50', 'C8','F11:J50', 'H8']
    else:
        return [] 
    

def process_excel_files(folder_path, file_names, sheet_name):
    summed_data = {}
    cell_file_count = {}

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        workbook = openpyxl.load_workbook(file_path, data_only=True)

        version = checkVersion(workbook)
        if not version or version < 2.4 or sheet_name not in workbook.sheetnames:
            continue

        sheet = workbook[sheet_name]
        range_ver = get_ranges_for_version(version)
        data = get_data_from_ranges(sheet, range_ver)

        df = pd.DataFrame(data)
        for _, row in df.iterrows():
            key = row['Cell']
            if key not in summed_data:
                summed_data[key] = {'Count_values': 0}
                cell_file_count[key] = 0

            summed_data[key]['Count_values'] += row['Count_values']
            cell_file_count[key] += 1

    result_data = {}
    for cell, values in summed_data.items():
        frequency = values['Count_values'] / cell_file_count[cell] if cell_file_count[cell] > 0 else 0
        result_data[cell] = round(frequency, 2)

    return result_data

In [ ]:
folder_path = os.path.join(os.getcwd(), "TEST_for_Infrastructure_Equipment")
sheet_name = 'Infrastructure_Equipment'
file_names = [f for f in os.listdir(folder_path)]
result_data = process_excel_files(folder_path, file_names, sheet_name)

c:\Users\iwachpul\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\reader\drawings.py:67: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
c:\Users\iwachpul\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
result_sheet = openpyxl.load_workbook('My_version_template.xlsx', keep_vba= False)
ws = result_sheet['Infrastructure_Equipment'] 

def get_topleft_of_merged_cells(sheet, coordinate):
    for merged_range in sheet.merged_cells.ranges:
        if coordinate in merged_range:
            # Return the top-left cell of this merged range
            return merged_range.coord.split(":")[0]
    return coordinate

# Write data back to the sheet
for key, value in result_data.items():
    top_left = get_topleft_of_merged_cells(ws, key)  # Get the top-left cell of the range if it's part of a merged cell
    ws[top_left] = value

result_sheet.save('My_version_template_saved.xlsx')